# Part 5

Import Libaries

In [1]:
import numpy as np
from collections import Counter, OrderedDict
import collections
import pandas as pd

/Users/tengfone/.pyenv/versions/3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Read File

In [2]:
def readFile(filePath): 
    with open (filePath, 'r') as f: 
        lines = f.readlines()
        text = []
        label = []
        label_split = []
        temp_arr = []
        label_split_temp = []
        for l in lines: 
            if l != "\n":
                l_split = l.strip().split(" ")
                te = " ".join(l_split[:-1])
                la = l_split[-1]
                temp_arr.append((te, la))
                label.append(l_split[-1])
                label_split_temp.append(l_split[-1])
            else: 
                text.append(temp_arr)
                temp_arr = []
                label_split.append(label_split_temp)
                label_split_temp = []
    return text, label, label_split 

Pre Processing of the loaded CSV File from the Part5_Preprocess.ipynb

In [3]:
def preprocess(filepath, lan):
    
    em_arr = pd.read_csv(filepath, sep=',',header=None)
    em_arr = em_arr.T
    em_arr = em_arr.drop([0, 2])
    em_arr.columns = em_arr.iloc[0]
    em_arr = em_arr[1:]
    if lan == 'EN':
        em_arr = em_arr.reindex([5,6,11,12,4,7,9,3,16,10,8,19,20,18,21,17,13,22,23,14,15])
    elif lan == 'CN':
        em_arr = em_arr.reindex([4,6,3,7,8,5,9])
    else: 
         em_arr = em_arr.reindex([6,3,4,7,5,9,8])
    header_list = list(em_arr.columns.values)
    em = em_arr.drop(['Text'], axis=1).values
    em = em.astype('float64')
    header_list = header_list[1:]
    return em, header_list

Getting Each Label Index

In [4]:
def get_label_index(filepath, header_list):
    test = []
    output_index = []
    output_label = []
    index = []
    label = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
        for l in lines: 
            if l != "\n":
                label.append(l.strip())
                try: 
                    index.append(header_list.index(l.strip()))
                except: 
                    index.append(header_list.index("#UNK#"))
            else: 
                output_index.append(index)
                output_label.append(label)
                index = []
                label = []
    return output_index, output_label


Convert Label to dictionary

In [5]:
def get_label_dict(text, label): 
    final_dict = {'START': len(text)}
    label_counter = dict(Counter(label))
    final_dict.update(label_counter)
    final_dict['STOP'] = len(text)
    return final_dict 

Getting Transmission Array

In [6]:
def get_tra_arr(d, a): 
    for seq in a:
        seq.insert(0, "START")
        seq.append("STOP")
    first_tra_arr = []
    for ci, c in enumerate(list(d.keys())[1:-1]):
        count_c = 0
        for seq in a:
            if seq[1] == c:
                count_c += 1
        first_tra_arr.append(count_c / len(a))
    tra_arr = []
    for wi, w in enumerate(list(d.keys())[:-1]):
        arr2 = []
        for ui, u in enumerate(list(d.keys())[1:-1]):
            arr1 = []
            count_wu = 0
            for seq in a:
                for c in range(len(seq)-1):
                    if seq[c] == w and seq[c+1] == u:
                        count_wu += 1
            for vi, v in enumerate(list(d.keys())[1:]):
                count_wuv = 0
                for seq in a:
                    for c in range(len(seq)-2):
                        if seq[c] == w and seq[c+1] == u and seq[c+2] == v:
                            count_wuv += 1
                if count_wu == 0:
                    arr1.append(-1)
                else:
                    arr1.append(count_wuv / count_wu)
            arr2.append(arr1)
        tra_arr.append(arr2)
    return first_tra_arr, tra_arr

Output File (Change output file name here)

In [7]:
def get_output(first_tra_arr, tra_arr, em, final_dict, index, label):
    seq = []
    for ci, i in enumerate(index): 
        seq_temp = vertibi_algo(first_tra_arr, tra_arr, em, final_dict, i)
        temp_arr = []
        for cj, j in enumerate(seq_temp): 
            temp_arr.append("{} {}".format(label[ci][cj],j))
        seq.append(temp_arr)

    # Change output file name
    with open ('test.p5.out', 'w') as f: 
        for i in seq: 
            for j in i: 
                f.write(j+"\n")
            f.write("\n")
    return "Done"

2nd Order HMM Algo

In [8]:
def vertibi_algo(first_tra_arr, tra_arr, em_arr, d, n): 
    # forward 
    rst = []
    pi_arr = []
    dy_arr = []
    maxidx_arr = []
    
    for ci, c in enumerate (n):
        if ci == 0:
            for i in range(len(em_arr)):
                dy_arr.append(first_tra_arr[i] * em_arr[i][n[ci]])
        else:
            temp_arr = []
            tempdy_arr = []
            tempmaxidx_arr = []
            for i in range(len(em_arr)): # for each node
                node_arr = []
                for j in range(len(em_arr)): # link to each previous node
                    if ci == 1:
                        node_arr.append(dy_arr[j] * tra_arr[ci-1][j][i] * em_arr[i][n[ci]])
                    else:
                        node_arr.append(dy_arr[j] * tra_arr[maxidx_arr[j]+1][j][i] * em_arr[i][n[ci]])
                temp_arr.append(node_arr)
                tempdy_arr.append(max(node_arr))
                tempmaxidx_arr.append(node_arr.index(max(node_arr)))
            pi_arr.append(temp_arr)
            dy_arr = tempdy_arr
            maxidx_arr = tempmaxidx_arr

    # backward
    maxidx = dy_arr.index(max(dy_arr))
    rst.append(list(d.keys())[maxidx+1])

    for i in range(len(n)+1):
        if len(pi_arr) != 0:
            pop_arr = pi_arr.pop(-1)
            check_arr = pop_arr[maxidx]
            maxidx = check_arr.index(max(check_arr))
            rst.append(list(d.keys())[maxidx+1])
    
    return list(reversed(rst))

In [9]:
# Change File Path Here 
pre_filepath = '/Users/tengfone/Desktop/ML_Project/Part5/TEST.csv'
devin_filepath = '/Users/tengfone/Desktop/ML_Project/data/test_data/test.in'
train_filepath = '/Users/tengfone/Desktop/ML_Project/data/EN(2)/train'

text, label, label_split = readFile(train_filepath)
final_dict = get_label_dict(text, label)
first_tra_arr, tra_arr = get_tra_arr(final_dict, label_split)
em, header_list = preprocess(pre_filepath, 'EN') # Indicate if it is EN/CN/SG
index, output_label = get_label_index(devin_filepath, header_list)

# Change output file name in get_output function
print(get_output(first_tra_arr, tra_arr, em, final_dict, index, output_label))

Done
